In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import os
import scipy.stats as stats
import random
from numpy import polyfit, poly1d

### 能否进入top (being top or not)

In [5]:
neuro = pd.read_csv('physics_0123.csv')
neuro = neuro[(neuro.StopYear<=1995)&(neuro.StopYear>=1960)]

In [6]:
interval = [[1960,1970],[1970,1980],[1980,1990],[1985,1995]]
Top = {}
for itl in interval:
    i0 = itl[0];i1 = itl[1]
    temp = np.array(neuro[['CID','YearlyCitationOfMentee']][(neuro.StopYear>=i0)&(neuro.StopYear<=i1)])
    CID = temp[:,0]
    YC = temp[:,1]
    mm = sorted(temp,key = lambda x:x[1],reverse=1)
    
    pt = 0.05   #进入 top 25
    th = mm[round(len(mm)*pt)][1]
    print(th)
    for i in range(len(CID)):
        if YC[i]>=th:
            Top[CID[i]]=1
        else:
            Top[CID[i]]=0

530.6346153846154
815.2439024390244
1014.1578947368421
971.516129032258


In [7]:
CID = neuro['CID']
b_or_s = []
for k in CID:
    b_or_s.append(Top[k])
neuro['Top25'] = b_or_s

##### 导出回归数据 (export regression data)

In [8]:
neuro_re = neuro[['MenteeID', 'MentorID','StopYear','BirthYearOfMentee','First5YearPubsOfMentee',
                  'First5YearCitationOfMentee','First5YearCollaOfMentee','CollaPubsOfWithMentor',
                  'BigOrSmall',
                  'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor','CareerAgeOfMentorInTraining',
                  'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','PubsOfMentorInTraning',
        
                  'Survival0','Survival1','YearlyCitationOfMentee','Top25','NumOfMenteesOfTheMenteeIn20']][(neuro.CareerAgeOfMentorInTraining<50)&(neuro.CareerAgeOfMentorInTraining>0)
                                             &(neuro.BigOrSmall!=-1)]
#neuro_era.BigOrSmall!=2)] #(neuro_era.TotalPubsOfMentor>0)&(neuro_era.CareerAgeOfMentorInTraining>-1)
                                    #(neuro_era.CareerAgeOfMentorInTraining<50)

In [9]:
neuro_re.shape

(3963, 22)

In [10]:
# neuro_re = neuro_re[(neuro_re.StopYear<=1995)&(neuro_re.StopYear>=1970)]

In [11]:
#测试大小团队中的生存率 (test survival rate)
all_big = neuro_re['MenteeID'][neuro_re.BigOrSmall==1]
all_small = neuro_re['MenteeID'][neuro_re.BigOrSmall==0]
sb = neuro_re['MenteeID'][neuro_re.Survival0==1]
sur_big = all_big[all_big.isin(sb)]
sur_small = all_small[all_small.isin(sb)]
print(len(sur_big)/len(all_big))
print(len(sur_small)/len(all_small))

0.40848094471282875
0.6780952380952381


In [12]:
neuro_re.to_csv('test0_0.25.csv',index=0)

##### 批量输出 cem 匹配截断点 (export cutting points)

In [13]:
cutpoint_list = ['First5YearPubsOfMentee','First5YearCitationOfMentee','First5YearCollaOfMentee',
                 'CollaPubsOfWithMentor',
                     'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor',
                     'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor',
                 'PubsOfMentorInTraning']

In [25]:
CutPoint = pd.DataFrame()
for k in cutpoint_list:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    # plt.hist(tp,range=[0,1000],bins=100)
    # print(a)
    print(k)
    qt = [0,0.3,0.6,1]
    cut = []
    for q in qt:
        cut.append(np.quantile(tp,q))
    CutPoint[k] = cut    
    print(cut)

First5YearPubsOfMentee
[0, 1.0, 8.0, 525]
First5YearCitationOfMentee
[0, 6.0, 175.0, 38297]
First5YearCollaOfMentee
[0, 2.0, 8.0, 294]
CollaPubsOfWithMentor
[0, 0.0, 0.0, 95]
TotalPubsOfMentor
[0, 15.0, 151.0, 2127]
TotalCitationOfMentor
[0, 260.0, 4912.0, 226338]
TotalCollaOfMentor
[0, 13.0, 124.0, 2619]
YearlyPubsOfMentor
[0.0, 0.9523809523809523, 2.8480186480186487, 75.57142857142857]
YearlyCitationOfMentor
[0.0, 12.852555301296722, 96.7106060606061, 4330.234042553191]
YearllyCollaOfMentor
[0.0, 0.7540983606557377, 2.5508163265306125, 87.3]
PubsOfMentorInTraning
[0, 0.0, 15.0, 300]


In [26]:
#手动补充和修正0特多的变量的cutpoint (manually minor adjustment for cutting points)
cp = ['CollaPubsOfWithMentor']
for k in cp:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    tp = tp[tp>0]
    # print(a)
    print(k)
    qt = [0,0.5,1]
    cut = [0]
    for q in qt:
        cut.append(np.quantile(tp,q))
    print(cut)
    CutPoint[k] = cut
CutPoint['CareerAgeOfMentorInTraining'] =  [0,15,25,50]

# CutPoint['TotalPubsOfMentor'] =  [0, 1000, 6000, 20854.0, 309472]
# CutPoint['TotalCitationOfMentor'] =  [0, 100, 3000,309472]
# CutPoint['TotalCollaOfMentor'] =  [0, 50, 200, 526.0, 12568]
# CutPoint['YearllyCollaOfMentor'] =  [0, 2, 8, 88]
CutPoint['PubsOfMentorInTraning'] =  [0, 6,15,300]

CollaPubsOfWithMentor
[0, 1, 3.0, 95]


In [27]:
CutPoint['First5YearPubsOfMentee'] =  [0,5.0, 10.0, 435]
# CutPoint['First5YearCollaOfMentee'] =  [0, 4, 8.0, 18.0, 294]
CutPoint['First5YearCitationOfMentee'] =  [0, 50,518.7999999999993, 38297]


In [28]:
CutPoint

,First5YearPubsOfMentee,First5YearCitationOfMentee,First5YearCollaOfMentee,CollaPubsOfWithMentor,TotalPubsOfMentor,TotalCitationOfMentor,TotalCollaOfMentor,YearlyPubsOfMentor,YearlyCitationOfMentor,YearllyCollaOfMentor,PubsOfMentorInTraning,CareerAgeOfMentorInTraining
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0,0
1,5.0,50.0,2.0,1.0,15.0,260.0,13.0,0.952381,12.852555,0.754098,6,15
2,10.0,518.8,8.0,3.0,151.0,4912.0,124.0,2.848019,96.710606,2.550816,15,25
3,435.0,38297.0,294.0,95.0,2127.0,226338.0,2619.0,75.571429,4330.234043,87.300000,300,50


In [29]:
CutPoint.to_csv('CutPoint0_0.25.csv',index=0)

In [226]:
# neuro_re1 = neuro_era[['NumOfMenteesOfTheMentor10','BirthYearOfMentee','First5YearPubsOfMentee','First5YearCitationOfMentee','First5YearCollaOfMentee',
#                      'BigOrSmall','TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor',
#                      'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','CareerAgeOfMentorInTraining',
#                     'Survival','NumOfMenteesOfTheMenteeIn20']][(neuro_era.CareerAgeOfMentorInTraining>-1)&(neuro_era.CareerAgeOfMentorInTraining<50)&
#                                 (neuro_era.TotalPubsOfMentor>0)&(neuro_era.BigOrSmall!=1)&(neuro_era.Survival==1)]

In [128]:
neuro.columns

Index(['CID', 'MenteeID', 'MentorID', 'MentorshipType', 'Institution',
       'InstitutionMAGID', 'StartYear', 'StopYear', 'NumOfMenteesOfTheMentor',
       'NumOfMenteesOfTheMentor10', 'NumOfMenteesOfTheMenteeIn20',
       'NumOfMenteesOfTheMentee', 'BigOrSmall', 'Survival0', 'Survival1',
       'TotalPubsOfMentor', 'YearOfTheFirstPubsOfMentorPubs',
       'YearlyPubsOfMentor', 'TotalCitationOfMentor', 'YearlyCitationOfMentor',
       'TotalCollaOfMentor', 'YearllyCollaOfMentor',
       'YearOfTheFirstPubsOfMenteePubs', 'First5YearPubsOfMentee',
       'First5YearCitationOfMentee', 'First5YearCollaOfMentee',
       'YearlyPubsOfMentee', 'YearlyCitationOfMentee', 'BirthYearOfMentee',
       'CareerAgeOfMentorInTraining', 'Top25'],
      dtype='object')

In [30]:
#限制条件比较少的情况 (less limitations)
neuro_re1 = neuro[['MenteeID', 'MentorID','StopYear','BirthYearOfMentee','First5YearPubsOfMentee',
                  'First5YearCitationOfMentee','First5YearCollaOfMentee','CollaPubsOfWithMentor',
                  'BigOrSmall',
                  'TotalPubsOfMentor','TotalCitationOfMentor','TotalCollaOfMentor','CareerAgeOfMentorInTraining',
                  'YearlyPubsOfMentor','YearlyCitationOfMentor','YearllyCollaOfMentor','PubsOfMentorInTraning',
        
                  'Survival0','Survival1','YearlyCitationOfMentee','Top25',
                   'NumOfMenteesOfTheMenteeIn20']][(neuro.CareerAgeOfMentorInTraining<50)&(neuro.CareerAgeOfMentorInTraining>0)
                                             &(neuro.BigOrSmall!=-1)&(neuro.Survival0==1)]
    

In [31]:
neuro_re1.shape

(2185, 22)

In [32]:
CutPoint1 = pd.DataFrame()
for k in cutpoint_list:
    tp = np.array(neuro_re1[k])
    # plt.hist(tp,range=[0,1000],bins=100)
    # print(a)
    print(k)
    qt = [0,0.3,0.6,1]
    cut = []
    for q in qt:
        cut.append(np.quantile(tp,q))
    CutPoint1[k] = cut    
    print(cut)

First5YearPubsOfMentee
[0, 4.0, 10.0, 525]
First5YearCitationOfMentee
[0, 48.0, 276.0, 38297]
First5YearCollaOfMentee
[0, 4.0, 10.0, 196]
CollaPubsOfWithMentor
[0, 0.0, 0.0, 41]
TotalPubsOfMentor
[1, 75.0, 209.0, 1445]
TotalCitationOfMentor
[0, 1952.7999999999993, 8120.0, 222046]
TotalCollaOfMentor
[0, 59.0, 165.39999999999986, 2272]
YearlyPubsOfMentor
[0.060606060606060615, 1.5516908212560383, 3.642283298097249, 42.411764705882355]
YearlyCitationOfMentor
[0.0, 44.59760368663595, 153.5818181818181, 4330.234042553191]
YearllyCollaOfMentor
[0.0, 1.2594074074074073, 3.153747645951036, 50.488888888888894]
PubsOfMentorInTraning
[0, 8.0, 22.0, 300]


In [33]:
#手动补充和修正0特多的变量的cutpoint (manually minor chage for cutting points)
cp = ['CollaPubsOfWithMentor']
for k in cp:
#     tp = np.array(neuro_re[k])
    tp = np.array(neuro[k])
    tp = tp[tp>0]
    # print(a)
    print(k)
    qt = [0,0.5,1]
    cut = [0]
    for q in qt:
        cut.append(np.quantile(tp,q))
    print(cut)
    CutPoint1[k] = cut
CutPoint1['CareerAgeOfMentorInTraining'] =  [0,15,25,50]

# CutPoint['TotalPubsOfMentor'] =  [0, 1000, 6000, 20854.0, 309472]
CutPoint1['TotalCitationOfMentor'] =  [0, 1000, 5000,309472]
# CutPoint['TotalCollaOfMentor'] =  [0, 50, 200, 526.0, 12568]
# CutPoint['YearllyCollaOfMentor'] =  [0, 2, 8, 88]
# CutPoint['PubsOfMentorInTraning'] =  [0, 6,19,300]

CollaPubsOfWithMentor
[0, 1, 3.0, 95]


In [34]:
neuro_re1.to_csv('test1_0.25.csv',index=0)

In [35]:
CutPoint1.to_csv('CutPoint1_0.25.csv',index=0)